<a href="https://colab.research.google.com/github/lsloan/udp-notebooks/blob/main/UDP_Connect_using_Credentials_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UDP: Connect Using a JSON Credentials File

## Introduction

Usually, Jupyter Notebooks documents in Google Colaboratory can access Google BigQuery databases with little effort using the "`bigquery`" Python module.  Using that, you can access any DB for which access has been granted to your individual Google account.

However, UDP's DB in BigQuery is different.  Unizin does not grant access to individual Google accounts.  Instead, they issue service account credentials personalized for each user.  These credentials are in the form of JSON files.  They are usually named something like "<code>**_YOUR_ORGNAME_HERE_**-**_YOUR_USERNAME_HERE_**-**_PROD_TEST_HERE_**.json</code>"

To use this notebook for querying Caliper events from UDP's BigQuery DB, you will need to upload your credentials JSON file into the "Files" section in the left pane of the Colab UI with the name "<code>**credentials.json**</code>".

ℹ️ Note: Items in the "Files" section of this notebook may be lost each time the runtime interpreter is restarted.  If you need to restart this notebook, you will also need to upload your "<code>**credentials.json**</code>" file again before you can connect to the DB and run queries.

## Initialize Environment

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Enable Google Colab option for Pandas dataframes pretty print
# (Use "reload_ext" instead of "load_ext" to avoid warning about reloading.)
%reload_ext google.colab.data_table

# Declare variable types and initial values
credentialsFileName: str = 'credentials.json'
credentials: service_account.Credentials = None
client: bigquery.Client = None

## Prepare Credentials

In [ ]:
# Reads credential file in the "File" section of this Google Colab notebook
credentials = service_account.Credentials.\
    from_service_account_file(credentialsFileName)

credentials

## Connect to DB

In [ ]:
client = bigquery.Client(
        credentials=credentials,
        project=credentials.project_id,
    )

project = client.project

print(f'Connected to BigQuery project "{project}".')
client

## Test Query

Try a query to show the connection to the DB is ready to use.

💡 Note: In the SQL query below, the variable "`project`" (defined in a code cell above), can be used to refer to the project of your DB.

### Most Recent Event by Store Time

Each event UDP stores includes the time it was stored in a column called "`store_time`".  Each event is *highly likely* to have a unique value in this column because it has nanosecond precision.  By getting the maximum value of that column, we can use it to find the one event that was most recently stored.


In [ ]:
recentEventStoredQuery: str = f'''
    SELECT
        id, event_time, store_time, ed_app, `type`, `action`
    FROM
        `{project}`.event_store.events
    WHERE
        store_time = (
            SELECT max(store_time) FROM `{project}`.event_store.events
        )
    ORDER BY
        store_time
'''

dfRecentEventStored: pd.DataFrame = \
    client.query(recentEventStoredQuery).to_dataframe()

dfRecentEventStored

# Next: Timestamp Differences

In the next notebook, I'll expand on the query by latest "`store_time`" and compare that with a query by latest "`event_time`".